- Определяем тиер в зависимости от ранга команды в мировом рейтинге, мол тиер -1 первые 15-25 команды, к примеру и выводим результат
- Важность турнира пока не знаю как вывести, но можно собрать инфу по количеству выигрываемых денег а также по тому, какие команды играют
- Рейтинг игроков я думаю пока средний 2.0 выводить
- Активность команды выводим по количеству сыгранных игр 
- История встреч, сыгранные карты и счёт на них
- История игр с одинаковыми врагами (типа вп - фазе, ликвид фазе, когда сравниваем вп и ликвид)
- Мапы пока думаю просто выводить списком, которые сортируется, мол сыграли с тир-1 командой, тир-2 и так далее командами, результат, противник, карту (тоже сортируем) и метку поражения или победы

In [60]:
import requests
import re

# получаем страницу игры по ссылке и выбираем карту (с маленькой буквы)
maps = 'mirage'
link = 'https://www.hltv.org/matches/2337357/g2-vs-nip-esl-pro-league-season-10-europe'
request = requests.get(link).text

# функция по определению тира команды
pattern_findrank = r"<a href=\"/ranking/teams\">#([\w-]+)<"
def get_tier(team_page):
    rank = re.findall(pattern_findrank, team_page)
    if len(rank) == 0:
        return 3
    else:
        if int(rank[0]) <= 15:
            return 1
        elif int(rank[0]) <=30:
            return 2
        else:
            return 3
    
# получаем название и страницу турнира
pattern_find_tournament = r"<div class=\"event text-ellipsis\"><a href=\"([\w/-]+)\" title=\"([\w\s/-]+)\""
tournament = re.findall(pattern_find_tournament, request)
tournament_link = 'https://www.hltv.org' + tournament[0][0]
tournament_name = tournament[0][1]
tournament_page = requests.get(tournament_link).text
print('Tournament:', tournament_name)

# получаем информацию о награде турнира
pattern_prize = r"<td class=\"prizepool text-ellipsis\" title=\"([\w\s$,+\-]+)\""
prize = re.findall(pattern_prize, tournament_page)[0]
print('Prize: ', prize)

# получаем тир турнира
pattern_tournament_teams = r"<a href=\"/team([\w/-]+)\" data-link-tracking-page=\"Eventpage\""
tournament_teams = re.findall(pattern_tournament_teams, tournament_page)
sum_rank = 0
for i in tournament_teams:
    page_team = requests.get('https://www.hltv.org/team' + i).text
    sum_rank += get_tier(page_team)
tournament_tier = sum_rank / len(tournament_teams)
print('Teams:', len(tournament_teams))
print('Tournament Tier:', round(tournament_tier, 1))

# получаем информацию о стадии турнира
pattern_tournament_stage = r"<div class=\"padding preformatted-text\">[\w\s]+\* ([\w\s]+)<"
tournament_stage = re.findall(pattern_tournament_stage, request)[0]
print('Tournament Stage:', tournament_stage)

# получаем название команд 
pattern_findteam = r"class=\"teamName\">([\w\s\.-]+)<"
teams = re.findall(pattern_findteam, request)
team1 = teams[0]
team2 = teams[1]

# получаем страницы команд
pattern_team1 = r"<div class=\"team1-gradient\"><a href=\"([\w/-]+)\""
team1_link = 'https://www.hltv.org' + re.findall(pattern_team1, request)[0] 
request1 = requests.get(team1_link).text
pattern_team2 = r"<div class=\"team2-gradient\"><a href=\"([\w/-]+)\""
team2_link = 'https://www.hltv.org' + re.findall(pattern_team2, request)[0]
request2 = requests.get(team2_link).text

# получаем тир команд
rank1 = get_tier(request1)
rank2 = get_tier(request2)

# получаем винрейт на картах
'''pattern_map = r"<img src=\"/img/static/maps/(\w+)\.png\" class=\"gtSmartphone-only\">"
maps = re.findall(pattern_map, request)
pattern_score = r"\[Map stats\] \[text\]\">([\w%-]+)<"
score = re.findall(pattern_score, request)
winrate_all_team = {}
i = 0
while i < len(maps):
    winrate_all_team[maps[i]] = (score[i*2], score[i*2+1])
    i += 1'''

# получаем ссылки на сыгранные карты за последние 3 месяца
pattern_stats_team = r"<a href=\"(/stats/teams/map/.*)\" class=\"moreButton\""
link_maps1 = re.findall(pattern_stats_team, request1)
link_maps2 = re.findall(pattern_stats_team, request2)

# получаем страницу с искомой картой
card_code = {'dust2':'/31/', 'mirage':'/32/', 'inferno':'/33/', 'Nuke':'/34/', 'train':'/35/', 'overpass':'/40/', 'vertigo':'/46/'}
pattern_code = r"(/\d\d/)"
score_link = 'https://www.hltv.org' + link_maps1[0]
score_link1 = re.sub(pattern_code, card_code[maps], score_link)
score_link1 = score_link1.replace("amp;", "")
score_page1 = requests.get(score_link1).text
score_link = 'https://www.hltv.org' + link_maps2[0]
score_link2 = re.sub(pattern_code, card_code[maps], score_link)
score_link2 = score_link2.replace("amp;", "")
score_page2 = requests.get(score_link2).text

# получаем статистику по сыгранной карте
pattern_times_played = r"<div class=\"stats-row\"><span class=\"strong\">Times played</span><span>(\w*)<"
times_played1 = re.findall(pattern_times_played, score_page1)[0]
times_played2 = re.findall(pattern_times_played, score_page2)[0]
pattern_wins_draws_losses = r"<div class=\"stats-row\"><span class=\"strong\">Wins / draws / losses</span><span>(\w*) / (\w*) / (\w*)<"
wins_draws_losses1 = re.findall(pattern_wins_draws_losses, score_page1)
wins_draws_losses2 = re.findall(pattern_wins_draws_losses, score_page2)
pattern_total_rounds_played = r"<div class=\"stats-row\"><span class=\"strong\">Total rounds played</span><span>(\w*)<"
total_rounds_played1 = re.findall(pattern_total_rounds_played, score_page1)[0]
total_rounds_played2 = re.findall(pattern_total_rounds_played, score_page2)[0]
patter_rounds_won = r"<div class=\"stats-row\"><span class=\"strong\">Rounds won</span><span>(\w+)<"
rounds_won1 = re.findall(patter_rounds_won, score_page1)[0]
rounds_won2 = re.findall(patter_rounds_won, score_page2)[0]
pattern_win_percent = r"<div class=\"stats-row\"><span class=\"strong\">Win percent</span><span class=\"\w*\">([\w\.%]*)<"
win_percent1 = re.findall(pattern_win_percent, score_page1)[0]
win_percent2 = re.findall(pattern_win_percent, score_page2)[0]
win_round_percent1 = str(round((int(rounds_won1) / int(total_rounds_played1) * 100), 1)) + '%'
win_round_percent2 = str(round((int(rounds_won2) / int(total_rounds_played2) * 100), 1)) + '%'
pattern_distribution_of_won_rounds = r"data&quot;:\[\{&quot;label&quot\;\:&quot;\w*&quot;,&quot;value&quot;:&quot;(\w*)&quot;,&quot;color&quot;:&quot;2C6EA4&quot;\},\{&quot;label&quot;:&quot;\w*&quot;,&quot;value&quot;:&quot;(\w*)&quot;"
distribution_of_won_rounds1 = re.findall(pattern_distribution_of_won_rounds, score_page1)
distribution_of_won_rounds2 = re.findall(pattern_distribution_of_won_rounds, score_page2)
win_ct_1 = str(round((int(distribution_of_won_rounds1[0][0]) / (int(distribution_of_won_rounds1[0][0]) + int(distribution_of_won_rounds1[0][1])) * 100), 2)) + '%' 
win_t_1 = str(round((int(distribution_of_won_rounds1[0][1]) / (int(distribution_of_won_rounds1[0][0]) + int(distribution_of_won_rounds1[0][1])) * 100), 2)) + '%' 
win_ct_2 = str(round((int(distribution_of_won_rounds2[0][0]) / (int(distribution_of_won_rounds2[0][0]) + int(distribution_of_won_rounds2[0][1])) * 100), 2)) + '%' 
win_t_2 = str(round((int(distribution_of_won_rounds2[0][1]) / (int(distribution_of_won_rounds2[0][0]) + int(distribution_of_won_rounds2[0][1])) * 100), 2)) + '%' 
pattern_pistol_rounds = r"<div class=\"stats-row\"><span class=\"strong\">Pistol rounds</span><span>(\w*)<"
pistol_rounds1 = re.findall(pattern_pistol_rounds, score_page1)[0]
pistol_rounds2 = re.findall(pattern_pistol_rounds, score_page2)[0]
pattern_pistol_rounds_won = r"<div class=\"stats-row\"><span class=\"strong\">Pistol rounds won</span><span>(\w*)<"
pistol_rounds_won1 = re.findall(pattern_pistol_rounds_won, score_page1)[0]
pistol_rounds_won2 = re.findall(pattern_pistol_rounds_won, score_page2)[0]
pattern_pistol_round_win_percent = r"<div class=\"stats-row\"><span class=\"strong\">Pistol round win percent</span><span class=\"\w*\">([\w\.%]*)<"
pistol_round_win_percent1 = re.findall(pattern_pistol_round_win_percent, score_page1)[0]
pistol_round_win_percent2 = re.findall(pattern_pistol_round_win_percent, score_page2)[0]

print()
print(team1)
print('Tier:', rank1)
print('----------------')
print(maps, 'stats')
print('Times played:', times_played1)
print('Wins/draws/losses:', wins_draws_losses1[0][0], wins_draws_losses1[0][1], wins_draws_losses1[0][2])
print('Total rounds played:', total_rounds_played1)
print('Rounds won:', rounds_won1)
print('Win percent:', win_percent1)
print('Win round percent:', win_round_percent1)
print('Win CT percent:', win_ct_1)
print('Win T percent:', win_t_1)
print('Pistol rounds:', pistol_rounds1)
print('Pistol rounds won:', pistol_rounds_won1)
print('Pistol round win percent:', pistol_round_win_percent1)

print()
print(team2)
print('Tier:', rank2)
print('----------------')
print(maps, 'stats')
print('Times played:', times_played2)
print('Wins/draws/losses:', wins_draws_losses2[0][0], wins_draws_losses2[0][1], wins_draws_losses2[0][2])
print('Total rounds played:', total_rounds_played2)
print('Rounds won:', rounds_won2)
print('Win percent:', win_percent2)
print('Win round percent:', win_round_percent2)
print('Win CT percent:', win_ct_2)
print('Win T percent:', win_t_2)
print('Pistol rounds:', pistol_rounds2)
print('Pistol rounds won:', pistol_rounds_won2)
print('Pistol round win percent:', pistol_round_win_percent2)

Tournament: ESL Pro League Season 10 Europe
Prize:  TBA
Teams: 16
Tournament Tier: 1.7
Tournament Stage: Group A

G2
Tier: 1
----------------
mirage stats
Times played: 4
Wins/draws/losses: 2 0 2
Total rounds played: 118
Rounds won: 58
Win percent: 50.0%
Win round percent: 49.2%
Win CT percent: 44.83%
Win T percent: 55.17%
Pistol rounds: 8
Pistol rounds won: 5
Pistol round win percent: 62.5%

NiP
Tier: 1
----------------
mirage stats
Times played: 7
Wins/draws/losses: 4 0 3
Total rounds played: 205
Rounds won: 105
Win percent: 57.1%
Win round percent: 51.2%
Win CT percent: 52.38%
Win T percent: 47.62%
Pistol rounds: 14
Pistol rounds won: 6
Pistol round win percent: 42.9%
